In [1]:
import numpy as np
import pandas as pd

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

In [5]:
!wget $data

--2025-10-14 15:11:05--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
 [text/plain] (79K)
Saving to: ‘course_lead_scoring.csv.1’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.02s   

2025-10-14 15:11:05 (3.37 MB/s) - ‘course_lead_scoring.csv.1’ saved [80876/80876]



In [8]:
df = pd.read_csv('course_lead_scoring.csv.1')

In [9]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [12]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [19]:
categorical = ['lead_source', 'industry', 'employment_status','location']
numerical = ['number_of_courses_viewed', 'annual_income','interaction_count','lead_score']

In [10]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [18]:
df.nunique()

lead_source                    5
industry                       7
number_of_courses_viewed      10
annual_income               1267
employment_status              4
location                       7
interaction_count             12
lead_score                   101
converted                      2
dtype: int64

In [20]:
for c in categorical:
    df[c] = df[c].fillna('NA')
for n in numerical:
    df[n] = df[n].fillna(0)

In [21]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [22]:
df.industry.value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

In [27]:
corr_matrix = round(df[numerical].corr(),2)
corr_matrix

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.00,0.01,-0.02,-0.00
annual_income,0.01,1.00,0.03,0.02
interaction_count,-0.02,0.03,1.00,0.01
lead_score,-0.00,0.02,0.01,1.00


## split data

In [30]:
pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 18.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 33.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [31]:
from sklearn.model_selection import train_test_split

In [33]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [35]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [37]:
len(df), len(df_train), len(df_val), len(df_test)

(1462, 876, 293, 293)

In [38]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [40]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [41]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

## mutual information

In [42]:
from sklearn.metrics import mutual_info_score

In [53]:
def mutual_info_convert_score(series):
    return mutual_info_score(series, df_full_train.converted)

In [54]:
mi = df_full_train[categorical].apply(mutual_info_convert_score)
mi

lead_source          0.025665
industry             0.011685
employment_status    0.013258
location             0.002253
dtype: float64

## One-hot encoding

In [56]:
from sklearn.feature_extraction import DictVectorizer

In [57]:
dv = DictVectorizer(sparse = False)

In [58]:
train_dict = df_train[categorical + numerical].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient = 'records')
X_val = dv.transform(val_dict)

## Training logistic regression

In [59]:
from sklearn.linear_model import LogisticRegression

In [60]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [61]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [63]:
y_val_pred = model.predict(X_val)

In [64]:
y_val_pred

array([1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0])

In [73]:
(y_val == y_val_pred).mean()

np.float64(0.6996587030716723)

## Feature elimination

In [67]:
features = categorical + numerical

In [74]:
for f in features:
    print(f)
    f_index = features.index(f)
    X_train_new = np.delete(X_train, f_index, axis=1)
    X_val_new = np.delete(X_val, f_index, axis=1)
    model.fit(X_train_new, y_train)
    y_val_pred = model.predict(X_val_new)
    print(round((y_val == y_val_pred).mean(),2) - (y_val == y_val_pred).mean())
    print()
    print()

lead_source
-0.0032423208191126873


industry
0.0003412969283276279


employment_status
0.0003412969283276279


location
0.0003412969283276279


number_of_courses_viewed
0.0003412969283276279


annual_income
0.003754266211604018


interaction_count
0.0003412969283276279


lead_score
0.0003412969283276279




## Parameter tuning

In [98]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

C = [0.01, 0.1, 1, 10, 100]
for c in C:
    print(c)
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train_scaled, y_train)
    y_val_pred = model.predict(X_val_scaled)
    print(round((y_val == y_val_pred).mean(),3))

0.01
0.843
0.1
0.85
1
0.853
10
0.853
100
0.853
